You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [16]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [17]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [18]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [19]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [20]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [24]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-11-21 11:45:22,851 - 139982012722048 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Topic: Artificial Intelligence

Outline:

I. Introduction
- Brief overview of Artificial Intelligence (AI)
- Importance of AI in various industries
- Thesis statement on the impact of AI on society

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI ethics and regulations updates
- AI in healthcare and finance sectors

III. Key Players in the AI Industry
- Leading tech companies investing in AI
- Startups disrupt

I now can give a great answer

Final Answer:
# The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has undoubtedly transformed the way we live and interact with technology in today's society. From self-driving cars to personalized recommendations on streaming platforms, AI has become an integral part of our daily lives, driving innovation and efficiency across various industries. It is crucial to understand the profound implications of AI as it continues to shape our world.

The latest trends in AI are characterized by rapid advancements in machine learning and natural language processing, allowing AI systems to analyze vast amounts of data and provide more accurate predictions and insights. However, the ethical considerations surrounding AI have become increasingly important, sparking conversations about regulations and guidelines to ensure responsible use of this powerful technology. In sectors like healthcare and finance, AI applications are revolutionizin

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has undoubtedly transformed the way we live and interact with technology in today's society. From self-driving cars to personalized recommendations on streaming platforms, AI has become an integral part of our daily lives, driving innovation and efficiency across various industries. It is crucial to understand the profound implications of AI as it continues to shape our world.

The latest trends in AI are characterized by rapid advancements in machine learning and natural language processing, allowing AI systems to analyze vast amounts of data and provide more accurate predictions and insights. However, the ethical considerations surrounding AI have become increasingly important, sparking conversations about regulations and guidelines to ensure responsible use of this powerful technology. In sectors like healthcare and finance, AI applications are revolutionizing service delivery and improving outcomes for both patients and customers.

Leading tech giants like Google, Amazon, and Microsoft are heavily investing in AI research and development, driving innovation and pushing the boundaries of what is achievable with AI. Additionally, startups are disrupting the AI market with niche solutions and cutting-edge technologies, while research institutions like MIT and Stanford are at the forefront of AI innovation, paving the way for future advancements in the field.

Recent breakthroughs in AI research have unlocked new possibilities for everyday applications, from virtual assistants to autonomous drones, reshaping how we interact with technology. Despite the exciting prospects AI brings, challenges such as bias in algorithms and data privacy concerns underscore the importance of ethical practices and regulations in the industry. It is essential for tech-savvy individuals, business professionals, students, and the general public to stay informed about the impact of AI on society, from understanding the latest trends to exploring ethical considerations and key players in the industry. Embracing ethical AI practices and staying up-to-date on advancements will be crucial as we navigate the evolving landscape of artificial intelligence.

*Stay informed about the latest developments in Artificial Intelligence by subscribing to our newsletter for regular updates and insights.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [25]:
topic = "Multi Agents Framwework"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Multi Agents Framwework.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: Exploring the Latest Trends in Multi Agents Framework

Introduction:
- Brief overview of Multi Agents Framework (MAF)
- Importance of MAF in various industries
- Mention of key players in the MAF market

Key Points:
1. Latest Trends in Multi Agents Framework:
- Use of MAF in autonomous vehicles and robotics
- Integration of MAF with IoT devices
- Advancements in MAF algorithms and technologies

2. Key Players in the MAF Market:
- OpenAI
- IBM Watson
- Google DeepMind

3. Notewort

I now can give a great answer

Final Answer:
# Exploring the Latest Trends in Multi Agents Framework

## Introduction to Multi Agents Framework
The Multi Agents Framework (MAF) is a cutting-edge technology that involves the coordination and communication of multiple autonomous agents to achieve a common goal. In various industries such as autonomous vehicles, robotics, and IoT devices, MAF plays a crucial role in enhancing efficiency and decision-making processes. Key players in the MAF market, including OpenAI, IBM Watson, and Google DeepMind, have been at the forefront of developing advanced algorithms and technologies to push the boundaries of multi-agent systems.

## Latest Trends in MAF
One of the latest trends in MAF is its integration with autonomous vehicles and robotics, where multiple agents work together seamlessly to navigate complex environments and make real-time decisions. Additionally, the integration of MAF with IoT devices has opened up new possibilities for smart, in

In [26]:
Markdown(result)

# Exploring the Latest Trends in Multi Agents Framework

## Introduction to Multi Agents Framework
The Multi Agents Framework (MAF) is a cutting-edge technology that involves the coordination and communication of multiple autonomous agents to achieve a common goal. In various industries such as autonomous vehicles, robotics, and IoT devices, MAF plays a crucial role in enhancing efficiency and decision-making processes. Key players in the MAF market, including OpenAI, IBM Watson, and Google DeepMind, have been at the forefront of developing advanced algorithms and technologies to push the boundaries of multi-agent systems.

## Latest Trends in MAF
One of the latest trends in MAF is its integration with autonomous vehicles and robotics, where multiple agents work together seamlessly to navigate complex environments and make real-time decisions. Additionally, the integration of MAF with IoT devices has opened up new possibilities for smart, interconnected systems that can adapt to changing conditions. Advancements in MAF algorithms and technologies have also led to more efficient and scalable solutions for a wide range of applications.

## Key Players in the MAF Market
OpenAI, IBM Watson, and Google DeepMind are among the key players in the MAF market, driving innovation and research in the field of multi-agent systems. These companies have been instrumental in developing state-of-the-art solutions that address the complexity of implementing MAF systems and provide valuable insights into the latest trends and advancements in the industry.

## Noteworthy News on Multi Agents Framework
Recent research studies on MAF applications have highlighted the practical applications of MAF in real-world scenarios, showcasing the potential for widespread adoption in various industries. Industry partnerships and collaborations in the MAF space have further accelerated the development of MAF technologies, leading to more robust and reliable systems. Upcoming events and conferences related to MAF provide opportunities for developers, engineers, business professionals, students, and researchers to stay updated on the latest trends and network with industry experts.

## Conclusion and Call to Action
In conclusion, the Multi Agents Framework is a dynamic and evolving technology that offers immense potential for innovation and growth in various industries. By staying informed about the latest trends, key players, and noteworthy news in the MAF market, individuals can gain valuable insights and stay ahead of the curve in implementing MAF solutions. I encourage all stakeholders in the AI and robotics field to explore the possibilities of MAF and leverage its capabilities to drive success in their respective domains.

Overall, this blog post aims to provide a comprehensive overview of the latest trends in Multi Agents Framework, highlighting the key players in the MAF market, and sharing noteworthy news on MAF applications. By addressing the pain points and interests of the target audience, this content piece seeks to educate and inform readers about the importance and potential of MAF in today's technology-driven world.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).